<a href="https://colab.research.google.com/github/ggongch2/web-scarping/blob/main/%EC%9B%B9%ED%81%AC%EB%A1%A4%EB%A7%81%EC%97%B0%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install selenium

In [ ]:
https://velog.io/@kite_day/colab-%EC%97%90%EC%84%9C-%EC%9B%B9-%ED%81%AC%EB%A1%A4%EB%A7%81%ED%95%98%EA%B8%B0-selenium

In [ ]:
from bs4 import BeautifulSoup, Tag
import requests
import re
import random
import sys
from selenium import webdriver

In [ ]:
url = "https://ko.wikipedia.org/wiki/%EC%9C%84%ED%82%A4%EB%B0%B1%EA%B3%BC:%EB%8C%80%EB%AC%B8"

driver = webdriver.Chrome()
driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
driver.save_screenshot("test1.png")


SessionNotCreatedException: Message: session not created: Chrome failed to start: exited normally.
  (session not created: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /root/.cache/selenium/chrome/linux64/131.0.6778.204/chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
#0 0x5a9d1a8618fa <unknown>
#1 0x5a9d1a372d20 <unknown>
#2 0x5a9d1a3aa438 <unknown>
#3 0x5a9d1a3a5f96 <unknown>
#4 0x5a9d1a3f1d86 <unknown>
#5 0x5a9d1a3f13d6 <unknown>
#6 0x5a9d1a3e5893 <unknown>
#7 0x5a9d1a3b430d <unknown>
#8 0x5a9d1a3b532e <unknown>
#9 0x5a9d1a82e00b <unknown>
#10 0x5a9d1a831f97 <unknown>
#11 0x5a9d1a81a71c <unknown>
#12 0x5a9d1a832b17 <unknown>
#13 0x5a9d1a7ff6cf <unknown>
#14 0x5a9d1a8506b8 <unknown>
#15 0x5a9d1a850880 <unknown>
#16 0x5a9d1a860776 <unknown>
#17 0x1324b3bcaac3 <unknown>


In [ ]:
def get_htmltext (url, tl) :
    response = requests.get(url, timeout = tl)
    if response.status_code == 200 :
        return response.text
    else :
        print(response.status_code)
        return None

# -> 셀레니움으로 하는 코드 만들어야함

In [ ]:
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.TAG_NAME, 'h1'))
)
